In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Load and preprocess data
df = pd.read_excel('Raw_Data_v0.xlsx', engine='openpyxl')
df = df.drop(columns=[
    'Ref#', 'Heat treatment', 'Other RM/Rivet/part cost (€/Part)',
    'Gross Weight (g)', 'Other assembled RM/Rivet/part',
])

num_cols = [
    'Annual target quantity', 'Raw Material Cost (€/kg)', 'Thickness (mm)',
    'Part Net Weight (g)', 'Surface Treatment cost (€/Part)',
    'Final Raw Material cost (€/Part)', 'Heat Treatment cost (€/Part)'
]
cat_cols = [
    'Production', 'Raw Material Designation',
    'Surface Treatment', 'Raw Material'
]

df[num_cols] = df[num_cols].fillna(0)
df[cat_cols] = df[cat_cols].fillna('Missing')
TARGET = 'Total cost with amortization (€/part)'

# Apply square root transformations
for col in num_cols + [TARGET]:
    df[col] = np.sqrt(df[col])

X = df[num_cols + cat_cols]
y = df[TARGET]
cat_indices = [X.columns.get_loc(col) for col in cat_cols]

# Best parameters with MAPE tracking
best_params = {
    'iterations': 786,
    'learning_rate': 0.0317,
    'depth': 5,
    'l2_leaf_reg': 2.216,
    'border_count': 83,
    'min_data_in_leaf': 34,
    'random_strength': 1.742,
    'subsample': 0.940,
    'bootstrap_type': 'Bernoulli',
    'feature_border_type': 'MaxLogSum',
    'grow_policy': 'SymmetricTree',
    'monotone_constraints': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
    'eval_metric': 'MAPE',
    'verbose': False
}

cost_components = [
    'Surface Treatment cost (€/Part)',
    'Final Raw Material cost (€/Part)',
    'Heat Treatment cost (€/Part)'
]

# 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
metrics = {
    'MAE': [], 'RMSE': [], 'MAPE': [], 'R2': [],
    'violations_before': [], 'violations_after': [],
    'within_10%': [], 'MoE_Improvement': []
}

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # 1. Data Encoding for Autoencoder
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    cat_train = ohe.fit_transform(X_train[cat_cols])
    cat_test = ohe.transform(X_test[cat_cols])
    
    encoded_train = np.hstack([X_train[num_cols].values, cat_train])
    encoded_test = np.hstack([X_test[num_cols].values, cat_test])
    
    # 2. Latent Space Construction
    input_dim = encoded_train.shape[1]
    latent_dim = max(4, min(8, input_dim//2))
    
    # Autoencoder architecture
    autoencoder = Sequential([
        Input(shape=(input_dim,)),
        Dense(64, activation='relu'),
        Dense(latent_dim, activation='relu', name='latent'),
        Dense(64, activation='relu'),
        Dense(input_dim, activation='linear')
    ])
    autoencoder.compile(optimizer=Adam(0.001), loss='mse')
    autoencoder.fit(encoded_train, encoded_train, 
                   epochs=50, batch_size=32, verbose=0)
    
    # 3. Expert Clustering in Latent Space
    latent_model = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('latent').output)
    latent_train = latent_model.predict(encoded_train)
    
    kmeans = KMeans(n_clusters=3, init='k-means++', n_init=20, random_state=42)
    train_clusters = kmeans.fit_predict(latent_train)
    
    # 4. Expert Model Training
    experts = []
    for cluster in range(3):
        mask = (train_clusters == cluster)
        if np.sum(mask) < 10:  # Skip small clusters
            continue
            
        # Use modified parameters for experts
        expert_params = best_params.copy()
        expert_params['iterations'] = max(100, best_params['iterations']//2)
        
        expert = CatBoostRegressor(**expert_params, cat_features=cat_indices)
        expert.fit(X_train[mask], y_train[mask])
        experts.append((expert, cluster))
    
    # 5. Baseline Model for Comparison
    baseline_model = CatBoostRegressor(**best_params, cat_features=cat_indices)
    baseline_model.fit(X_train, y_train)
    baseline_pred = baseline_model.predict(X_test)
    
    # 6. MoE Prediction
    test_latent = latent_model.predict(encoded_test)
    test_clusters = kmeans.predict(test_latent)
    
    y_pred = np.zeros_like(y_test)
    for expert, cluster in experts:
        mask = (test_clusters == cluster)
        if mask.any():
            y_pred[mask] = expert.predict(X_test[mask])
    
    # Handle unclustered samples with baseline
    unclustered_mask = ~np.isin(test_clusters, [c for _, c in experts])
    if np.any(unclustered_mask):
        y_pred[unclustered_mask] = baseline_model.predict(X_test[unclustered_mask])
    
    # 7. Evaluation
    y_pred_orig = np.square(y_pred)
    y_test_orig = np.square(y_test)
    baseline_pred_orig = np.square(baseline_pred)
    
    # Calculate component costs sum
    sum_costs = np.sum(np.square(X_test[cost_components]), axis=1)
    
    # Track violations
    violations_before = np.sum(y_pred_orig < sum_costs)
    y_pred_clipped = np.maximum(y_pred_orig, sum_costs)
    violations_after = np.sum(y_pred_clipped < sum_costs)
    
    # Store metrics
    metrics['MAE'].append(mean_absolute_error(y_test_orig, y_pred_clipped))
    metrics['RMSE'].append(np.sqrt(mean_squared_error(y_test_orig, y_pred_clipped)))
    metrics['MAPE'].append(np.mean(np.abs((y_test_orig - y_pred_clipped)/y_test_orig)) * 100)
    metrics['R2'].append(r2_score(y_test_orig, y_pred_clipped))
    metrics['violations_before'].append(violations_before)
    metrics['violations_after'].append(violations_after)
    metrics['within_10%'].append(np.sum(np.abs((y_pred_clipped - y_test_orig)/y_test_orig) <= 0.1))
    metrics['MoE_Improvement'].append(r2_score(y_test_orig, y_pred_clipped) - r2_score(y_test_orig, baseline_pred_orig))
    
    # Cluster analysis
    print(f"\n=== Fold {fold+1} Cluster Analysis ===")
    for cluster in range(3):
        if cluster in [c for _, c in experts]:
            cluster_data = X_train[train_clusters == cluster]
            print(f"Cluster {cluster} (n={len(cluster_data)})")
            print(cluster_data[cost_components].mean().sort_values(ascending=False))
    
    # Plot learning curve for last fold
    if fold == 9:
        plt.figure(figsize=(12, 6))
        
        # MoE vs Baseline comparison
        plt.scatter(y_test_orig, y_pred_clipped, alpha=0.5, label='MoE Predictions')
        plt.scatter(y_test_orig, baseline_pred_orig, alpha=0.5, label='Baseline Predictions')
        plt.plot([y_test_orig.min(), y_test_orig.max()], 
                [y_test_orig.min(), y_test_orig.max()], 'k--')
        plt.title('Prediction Comparison: MoE vs Baseline')
        plt.xlabel('True Values')
        plt.ylabel('Predictions')
        plt.legend()
        plt.show()

# Final results
print("\n=== Final Validation Results ===")
print(f"MAE: {np.mean(metrics['MAE']):.2f} ± {np.std(metrics['MAE']):.2f}")
print(f"RMSE: {np.mean(metrics['RMSE']):.2f} ± {np.std(metrics['RMSE']):.2f}")
print(f"MAPE: {np.mean(metrics['MAPE']):.2f}% ± {np.std(metrics['MAPE']):.2f}")
print(f"R²: {np.mean(metrics['R2']):.2f} ± {np.std(metrics['R2']):.2f}")
print(f"MoE R² Improvement: {np.mean(metrics['MoE_Improvement']):.3f} ± {np.std(metrics['MoE_Improvement']):.3f}")
print(f"Violations before clipping: {sum(metrics['violations_before'])} ({sum(metrics['violations_before'])/len(X)*100:.2f}%)")
print(f"Violations after clipping: {sum(metrics['violations_after'])} ({sum(metrics['violations_after'])/len(X)*100:.2f}%)")
print(f"Predictions within ±10%: {sum(metrics['within_10%'])/len(X)*100:.2f}%")


ModuleNotFoundError: No module named 'tensorflow.python'